In [ ]:
# Clone the entire repo.
!git clone -l -s git://github.com/aaron-xichen/pytorch-playground.git cloned-repo
%cd cloned-repo
!ls

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls /content/gdrive/My\ Drive/Colab\ Notebooks/*.py

In [ ]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks')

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jul  9 17:49:00 2019

@author: Zefyr Scott

Summary of files:
    * manager (you are here): Run everything from here. See below for a variety of
        configurable settings.
    * datasets: Dataset loading. Add additional datasets here, and note them in the
        AVAILABLE DATASET LIST in testmanager (below).
    * testandtrain: Handles training and testing processes
    * nnsupport: The model itself, as well as custom layers and supporting functions
    * wideresnet: A classifier called by nnsu
    
"""
import testandtrain
import torch
#import torchvision
#import torchvision.transforms as transforms
import supermodel
from datetime import datetime
import datasets
import tools
import smtplib



# Settings for various testing and troubleshooting
# Setting break_after_first = True runs only one batch, through training only.
break_after_first = False
# Setting train_only = True runs the training process only.
train_only = False
# Setting classify_only = True will skip the encoding/decoding steps
classify_only = False
# Setting print_hooks = True will print various info about gradients, weights,
# and biases
print_hooks = False
# Set this to true if using a Jupyter notebook
using_notebook = True
print_status = False

batch_size_train = 100
batch_size_test = 1000

num_epochs = 50
learning_rate = 0.001
momentum = 0.9 # used for SGD

### BIT SIZE STUFF ###
# Size of the encoder output. Use as number of bits if fitting all encoder data
# to [0,1]
encoded_size = 8

# Neuron info
# * MNIST "Simple 1D" encoder: around 200 is a good encoder hidden size.
# * CIFAR10 "FC for RGB" encoder: around 7000 is performing less badly than others
encoder_hidden_size = 200
classifier_hidden_size = 1000

# Dataset management
# AVAILABLE DATASET LIST: MNIST, CIFAR10
dataset = "CIFAR10"

# Splitters for different datasets:
# MNIST: 1D
# CIFAR10: FC for RGB, Color Separation
#   * FC for RGB: used with Simple 1D encoder and decoder
#   * Color Separation: used with Simple Conv encoder and decoder, and
#     num_of_encoders = 3. This handles each color layer as a distinct 2d image
#     for encoding and decoding
#   * 2D: splits along two different dimensions
splitter = "Color Separation"

# Encoders for different datasets:
# MNIST: Simple 1D
# CIFAR10: Simple 1D, 1 Channel Conv, 1 Channel Conv Square, AlexNetEncode, ResNetEncode
encoder = "ResNetEncode"

# Decoders for different datasets:
# MNIST: Simple 1D
# CIFAR10: Simple 1D, 1 to 3 Channel Conv, 1 to 3 Channel Conv Square
decoder = "1 to 3 Channel Conv"

# Classifiers for different datasets:
# Pretrained classifiers are from https://github.com/aaron-xichen/pytorch-playground
# MNIST: MNIST Tutorial, Pretrained
# CIFAR10: CIFAR10 Tutorial, Pretrained
classifier = "Pretrained"

# Load stuff for this dataset
loaders, classes, unencoded_dims, pretrained_classifier = datasets.loadDataset(dataset, encoder, classifier, batch_size_train, batch_size_test)

## For splitting along one dimension only:
## Set num_of_encoders to an integer to split the input automatically as evenly as
## possible. Alternately, if manual splitting is desired, ignore the num_of_encoders
## and instead directly assign a tuple to subsets indicating the desired splitting,
## ie if input size is 15 the desired splitting might be subsets = (8, 4, 3)
num_of_encoders = 3
subsets = tools.split_sizes(unencoded_dims['size'], num_of_encoders)
dims = 1
# For splitting across two different dimensions: if 4d RGB and splitting image with
# intact colors into even chunks, just specify the number of rows and columns
#rows = 2
#cols = 2
#subsets = (tools.split_sizes(unencoded_dims['height'], rows), tools.split_sizes(unencoded_dims['width'], cols))
#dims = (2,3)

# Loss function management
# AVAILABLE LOSS FUNCTION LIST: CrossEntropyLoss, CELWithEncodedMSE
loss_function = "CrossEntropyLoss"
#If using CELWithEncodedMSE, multiply the MSE part by this constant
mse_multiplier = 1

# General handling stuff
outfile = '/content/gdrive/My Drive/Colab Notebooks/{}.txt'.format(datetime.strftime(datetime.now(), "%y%m%d-%H%M%S"))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = "cpu"

#for i  in learning_rate_set:
#for i in range(1,5):
    #resead at start of each run for repeatability
torch.manual_seed(1)    
#outfile = None

with open(outfile, 'a') as outf:
    print(datetime.now().time(), 'Dataset: ', dataset, file=outf)
    print(datetime.now().time(), 'Classifier: ', classifier, file=outf)
    print(datetime.now().time(), 'Epochs: ', num_epochs, file=outf)
    print(datetime.now().time(), 'Learning rate: ', learning_rate, file=outf)
    print(datetime.now().time(), 'Loss function: ', loss_function, file=outf)
    print(datetime.now().time(), 'Optimizer: Adam', file=outf)
    if (classify_only):
        print(datetime.now().time(), 'Classification test only ', file=outf)
    else:
        if (encoder == '3 Channel Small Conv'):
            print(datetime.now().time(), 'Encoders: ', rows * cols, file=outf)
        else: print(datetime.now().time(), 'Encoders: ', num_of_encoders, file=outf)
        print(datetime.now().time(), 'Encoder type: ', encoder, file=outf)
        if (encoder == 'Simple 1D'):
            print(datetime.now().time(), 'Encoder neurons: ', encoder_hidden_size, file=outf)
        print(datetime.now().time(), 'Bits per encoder: ', encoded_size, file=outf)
        print(datetime.now().time(), 'Decoder type: ', decoder, file=outf)
        
unrounded_error, rounded_error = testandtrain.testandtrain(loaders,
            num_epochs,
            learning_rate,
            momentum,
            unencoded_dims,
            subsets,
            dims,
            mse_multiplier,
            encoded_size,
            classes,
            encoder_hidden_size,
            classifier_hidden_size,
            loss_function,
            splitter,
            encoder,
            decoder,
            classifier,
            outfile,
            break_after_first,
            train_only,
            classify_only, 
            print_hooks,
            using_notebook,
            device,
            pretrained_classifier,
            print_status)

with open(outfile, 'a') as outf:
    print(datetime.now().time(), 'Unrounded error: ', unrounded_error, '\n', file=outf)
    print(datetime.now().time(), 'Rounded error: ', rounded_error, '\n', file=outf)

# Stuff can take a long time to run so alert me when done
tools.alert()
